In [52]:
#IMPORTS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier

In [53]:
def load_data(df):
    y=df.quality
    X=df.drop('quality',1)
    return X, y

In [54]:
#CREATE AND MERGE DATAFRAMES
dfRed=pd.read_csv("winequality-red.csv", sep=";")
dfWhite=pd.read_csv("winequality-white.csv", sep=";")

dfRed['Type']=0 #"Red"
dfWhite['Type']=1 #"White"
df=pd.concat([dfRed,dfWhite])

dfAux=df.copy()
df.loc[dfAux['quality'] < 6, 'quality'] = "Baixa"
df.loc[dfAux['quality'] == 6, 'quality'] = "Mitjana"
df.loc[dfAux['quality'] > 6, 'quality'] = "Alta"
del dfAux


X, y = load_data(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

C:\Users\walli\AppData\Local\Temp/ipykernel_11584/4001844796.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X=df.drop('quality',1)


In [55]:
#LOGISTIC REGRESSION

logreg = LogisticRegression()
logreg.fit(X_train,y_train)

y_predict = logreg.predict(X_test)

dfLogReg=pd.DataFrame()
dfLogReg['Real'] = y_test
dfLogReg['Prediction']=y_predict
print(dfLogReg.head)

training_accuracy = logreg.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = logreg.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

#TODO: Perquè dona aquells warnings?

<bound method NDFrame.head of          Real Prediction
2173    Baixa    Mitjana
4410    Baixa    Mitjana
1865  Mitjana    Mitjana
1518    Baixa    Mitjana
2162  Mitjana      Baixa
...       ...        ...
2677  Mitjana    Mitjana
3683  Mitjana    Mitjana
2255  Mitjana      Baixa
1759    Baixa    Mitjana
4267     Alta       Alta

[1625 rows x 2 columns]>
training_accuracy: 0.5139573070607554
testing_accuracy: 0.4929230769230769


C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [56]:
#PERCEPTRON
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

y_predict=perceptron.predict(X_test)

dfPerceptron=pd.DataFrame()
dfPerceptron['Real'] = y_test
dfPerceptron['Prediction']=y_predict
print(dfPerceptron.head)

training_accuracy = perceptron.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = perceptron.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

<bound method NDFrame.head of          Real Prediction
2173    Baixa       Alta
4410    Baixa       Alta
1865  Mitjana       Alta
1518    Baixa       Alta
2162  Mitjana       Alta
...       ...        ...
2677  Mitjana       Alta
3683  Mitjana       Alta
2255  Mitjana       Alta
1759    Baixa       Alta
4267     Alta       Alta

[1625 rows x 2 columns]>
training_accuracy: 0.2243431855500821
testing_accuracy: 0.21353846153846154


In [57]:
#RANDOM FOREST
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_predict=rf.predict(X_test)

dfRF=pd.DataFrame()
dfRF['Real'] = y_test
dfRF['Prediction']=y_predict
print(dfRF.head)

training_accuracy = rf.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = rf.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

<bound method NDFrame.head of          Real Prediction
2173    Baixa      Baixa
4410    Baixa      Baixa
1865  Mitjana      Baixa
1518    Baixa    Mitjana
2162  Mitjana      Baixa
...       ...        ...
2677  Mitjana       Alta
3683  Mitjana    Mitjana
2255  Mitjana    Mitjana
1759    Baixa      Baixa
4267     Alta    Mitjana

[1625 rows x 2 columns]>
training_accuracy: 1.0
testing_accuracy: 0.7107692307692308
